# Hubert Stefanski - Data Mining II - Main Assignment
## 20081102@mail.wit.ie

## Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

from IPython.display import display, Markdown
plt.style.use("seaborn-darkgrid")
pd.set_option('display.max_columns', None)  

## Helper functions and globals
don't mind this, I'm just lazy

In [2]:

def name_func(df):
    col_names = []
    for i in range(df.shape[1]):
        col_names.append("Col_" + str(i))
    return col_names

SEED = 64
TARGET = "Target"

## Load Datasets

In [3]:
# Load and name columns, change last column name to "Target"
df_labeled = pd.read_csv("data/labeled.csv.gz",names = name_func(pd.read_csv("data/labeled.csv.gz")))
df_labeled.columns = [*df_labeled.columns[:-1],"Target"]
print("Labeled", df_labeled.shape)
# df_labeled.head()

# Load and name columns
df_scoring = pd.read_csv("data/scoring.csv.gz",names = name_func(pd.read_csv("data/scoring.csv.gz")))
print("Scoring", df_scoring.shape)
# df_scoring.head()
                         

Labeled (16000, 177)
Scoring (9000, 176)


## Cleanup

In [4]:
# Change categorical data to binary flag
for col in df_labeled.select_dtypes(include=['object']).copy():
    df_labeled[col] = df_labeled[col].str.lower()
    encoder = OrdinalEncoder(categories=[['no','yes']])
    df_labeled[col] = encoder.fit_transform(df_labeled.loc[:,[col]])

# Check whether NaNs exist
print("Nan exists?: " + str(df_labeled.isnull().values.any()))

# Check total number of NaNs
print("NaN count: " + str(df_labeled.isnull().sum().sum()))

# Cleanup NaNs
for col in df_labeled.columns:
    if df_labeled[col].isna().values.any():
        # Replace all NaNs with means of the column
        df_labeled[col] = df_labeled[col].fillna(df_labeled[col].mode().values[0])

# Check whether NaNs exist after nan Cleanup      
print("Nan exists?: " + str(df_labeled.isnull().values.any()))
# Check total number of NaNs
print("NaN count: " + str(df_labeled.isnull().sum().sum()))


Nan exists?: True
NaN count: 3365
Nan exists?: False
NaN count: 0


### Correcting dTypes

In [5]:
df_labeled.dtypes.value_counts()

float64    159
int64       18
dtype: int64

#### Comment:
no need to fix dtypes, all int64 and float64

### Categorical/Numerical split

In [6]:
# Get column names as list
attributes = df_labeled.columns.tolist()
# Remove the target column, no point in doing this if we include it
attributes.remove(TARGET)

temp = df_labeled[attributes].nunique()

categorical = list(temp[(temp>1)& (temp<20)].index)

numerical = list(temp[(temp>20)].index)

f"Categorical count: {len(categorical)} and \n Numerical count:{len(numerical)}"

'Categorical count: 29 and \n Numerical count:143'

## Output the cleaned up DataFrame to a fresh csv
This allows some insight through dTale

In [7]:
df_labeled.to_csv("output/clean_df_labeled.csv", index=False)

### Exploratory Data Analysis

### Model Building

In [8]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df_labeled,stratify= df_labeled[TARGET], test_size=.4, random_state=SEED)
print(df_train.shape,df_test.shape)

(9600, 177) (6400, 177)


In [9]:
df_train.head(1)

,Col_0,Col_1,Col_2,Col_3,Col_4,Col_5,Col_6,Col_7,Col_8,Col_9,Col_10,Col_11,Col_12,Col_13,Col_14,Col_15,Col_16,Col_17,Col_18,Col_19,Col_20,Col_21,Col_22,Col_23,Col_24,Col_25,Col_26,Col_27,Col_28,Col_29,Col_30,Col_31,Col_32,Col_33,Col_34,Col_35,Col_36,Col_37,Col_38,Col_39,Col_40,Col_41,Col_42,Col_43,Col_44,Col_45,Col_46,Col_47,Col_48,Col_49,Col_50,Col_51,Col_52,Col_53,Col_54,Col_55,Col_56,Col_57,Col_58,Col_59,Col_60,Col_61,Col_62,Col_63,Col_64,Col_65,Col_66,Col_67,Col_68,Col_69,Col_70,Col_71,Col_72,Col_73,Col_74,Col_75,Col_76,Col_77,Col_78,Col_79,Col_80,Col_81,Col_82,Col_83,Col_84,Col_85,Col_86,Col_87,Col_88,Col_89,Col_90,Col_91,Col_92,Col_93,Col_94,Col_95,Col_96,Col_97,Col_98,Col_99,Col_100,Col_101,Col_102,Col_103,Col_104,Col_105,Col_106,Col_107,Col_108,Col_109,Col_110,Col_111,Col_112,Col_113,Col_114,Col_115,Col_116,Col_117,Col_118,Col_119,Col_120,Col_121,Col_122,Col_123,Col_124,Col_125,Col_126,Col_127,Col_128,Col_129,Col_130,Col_131,Col_132,Col_133,Col_134,Col_135,Col_136,Col_137,Col_138,Col_139,Col_140,Col_141,Col_142,Col_143,Col_144,Col_145,Col_146,Col_147,Col_148,Col_149,Col_150,Col_151,Col_152,Col_153,Col_154,Col_155,Col_156,Col_157,Col_158,Col_159,Col_160,Col_161,Col_162,Col_163,Col_164,Col_165,Col_166,Col_167,Col_168,Col_169,Col_170,Col_171,Col_172,Col_173,Col_174,Col_175,Target
10116,60.0,22.7931,0.6356,0.000516,0,1.1447,0.1093,3,94.8683,0.0,0.0889,6,1.1989,0.0,60.0,4467.684,0.63,2,29.7,0.1289,12277.62,0.0,1.0,0.002672,90.0,1.0,0.5813,0.00002,0.0,3.0,0.0,0.63,0.81,11.61,0.0,0.000168,0.0,6.2984,3.0,27.3705,0.0,0.0,60.12,0.001472,0.0,3,0.0,0.0,24.669,26.1543,67.082,1,0.0424,1.1564,1.7731,2005,1.8,0.0756,838.4903,0.0,1.134,0.007164,67.082,2,0.6653,1.1493,19.98,1.1258,33.12,0.00008,0.45,15.698,0.7707,0.0,1.0,0.0,0.02838,9108.9,2.43,4.1,0.00132,4,0.5374,1407.35,0.0044,0.0657,5.28,0.0,425.192,0.0133,0.99,44.5266,5.5897,10.3882,5.0,1374.1179,1.0,0.40484,1,8.7115,0.0,1.048,1.064,0.0,3,1.064,0.224,1.0711,0.0,1.1327,11.7,34.38,638.55,0.0267,1.0861,1.0,0,2.5556,0.2012,161.19,3.78,0.545672,67.082,5,0.0613,2,8.1351,6.03,3.2308,6.4568,1.1176,5590.7155,45.688,49.9053,2.328,0.0,0.45,60.0,1.6222,1402,3.0,0.0,0.9,0.0222,0.000268,0.0,0.04,167.13,4.3777,0.007428,1.0,4.0,0.9525,0.0,0.72,60.0,1.0,3,0.72,0.0,0.0,3.3529,3.6522,0.0311,1.509,1.0699,0.0311,1.26,1.0,1.2,1.0993,0.016,2.0,1.1065,0.1413,4,0


#### Data normalization and scaling

In [10]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train = scaler.fit_transform(df_train.loc[:,attributes].astype(float))
y_train = df_train[TARGET].values

X_test = scaler.transform(df_test.loc[:,attributes].astype(float))
y_test = df_test[TARGET].values

### Model Baseline Score
Doing this just to get a rough idea of how well the model scores by itself without any new parameters or optimisation

In [11]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()

param_range = np.logspace(-4, 4, 9) 

param_grid = [{
    'solver': ['newton-cg', 'lbfgs', 'saga'],
     'C': param_range, 
    'penalty': ['l1', 'none']
}]

In [12]:
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
gs = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring='accuracy', error_score=0)

gs_result = gs.fit(X_train, y_train)

In [14]:
print("Best: %f using %s" % (gs_result.best_score_, gs_result.best_params_))

Best: 0.708958 using {'criterion': 'entropy', 'max_features': 9, 'n_jobs': -1, 'random_state': 3}


In [16]:
means = gs_result.cv_results_['mean_test_score']
stds = gs_result.cv_results_['std_test_score']
params = gs_result.cv_results_['params']

for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

0.702014 (0.015174) with: {'criterion': 'gini', 'max_features': 9, 'n_jobs': -1, 'random_state': 3}
0.705208 (0.016532) with: {'criterion': 'gini', 'max_features': 9, 'n_jobs': -1, 'random_state': 6}
0.708958 (0.014473) with: {'criterion': 'entropy', 'max_features': 9, 'n_jobs': -1, 'random_state': 3}
0.707951 (0.015175) with: {'criterion': 'entropy', 'max_features': 9, 'n_jobs': -1, 'random_state': 6}


### Feature Engineering

### Hyper Parameter Optimisation

### Scoring -- `accuracy`

### Scoring -- `rof_auc`